In [1]:
/*
Some key notes to read about this script:
This script is supposed to generate figures for the "content generation" part of the nanoHUB annual report. 
Four files were given for that purpose: db_connect.php, gen_table_content_generation.php, hub_parameters.php and func_misc.php
Because jupyter lab doesn't allow inclusion of php files directly, three out of the four files (db_connect.php, gen_table_content_generation.php, hub_parameters.php) 
contents are all in this script. 
db_connect had to be rewritten to connect through the ssh server remotely and is described by the following three cells)
the fourth cell has the contents of the hub_parameters.php file
the rest contains the contents originally in gen_table_content_generation.php
*/

In [2]:
function test_execute_query(mysqli $connection, string $query = "SELECT registerDate, username FROM nanohub.jos_users LIMIT 10;") 
{
    $result = $connection->query($query);
    /*
    while ($row = $result->fetch_row()) {
        print_r($row);
    }
    */
    return $result;
}

In [3]:
class ConnectionConfig 
{
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

In [4]:
interface IMySqliConnection
{
    public function new_connection(ConnectionConfig $mysqli_config);
}

class TunneledMySqliConnection implements IMySqliConnection
{
    public $ssh_config;
    public $local_port;
    
    public function __construct(
        ConnectionConfig $ssh_config, int $local_port = 3307
    )
    {
        $this->ssh_config = $ssh_config;
        $this->local_port = $local_port;
    }
    
    public function new_connection(ConnectionConfig $mysqli_config)
    {    
        
        $ssh_connection_string = sprintf(
            "sshpass -p '%s' ssh -vfN -o 'StrictHostKeyChecking=no' -L %s:%s:%d %s@%s ", 
            $this->ssh_config->password, 
            $this->local_port,
            $mysqli_config->hostname, 
            $mysqli_config->port,
            $this->ssh_config->username,
            $this->ssh_config->hostname
        );
        
        shell_exec($ssh_connection_string);
        usleep(1000);
        
        mysqli_report(MYSQLI_REPORT_ERROR | MYSQLI_REPORT_STRICT);
        return mysqli_connect(
            $mysqli_config->hostname, 
            $mysqli_config->username,
            $mysqli_config->password, 
            '',
            $this->local_port
        );
    }
}

class TunneledMySqliConnectionFactory
{
    static public function create_from_environment(int $local_port = 3307)
    {
        $ssh_config = new ConnectionConfig(
            getenv('tunnel_ssh_username'),
            getenv('tunnel_ssh_password'),
            getenv('tunnel_ssh_host'),
            getenv('tunnel_ssh_port')
        );
        
        $connection = new TunneledMySqliConnection($ssh_config, $local_port);
        
        $mysqli_config = new ConnectionConfig(
            getenv('db_user'),
            getenv('db_password'),
            getenv('db_host'),
            getenv('tunnel_remote_bind_port')
        );
        
        return $connection->new_connection($mysqli_config);
    }
    
}

In [5]:
// global variables needed for the other functions (comes from hub_parameters.ipynb)
$hub_db = '`nanohub`';
$hub_dir = '/www/nanohub';
$db_host = 'lightsout.nanohub.org';
$db_user = 'nanohub';
$db_pass = 'XXX';
$db_prefix = 'jos_';

$metrics_db = 'nanohub_metrics';
$report_db = 'nanohub_annualreport';
$mw_db = 'narwhal';
$net_db = 'network';

$db_net_host = 'lightsout.nanohub.org';
$db_net_user = 'geodb';
$db_net_pass = 'XXX';
$net_db = 'network';


"network"

In [6]:
// for the get NCN participation data 
function get_ncn_participation ($db_hub, $period, $label, $report_db) {

	global $report_db;
	$tag_list = '';
	if ($label == "NCN Purdue") {
		$sql = 'SELECT tagid FROM '.$report_db.'.report_ncn_tags WHERE report_tag IN (SELECT report_tag FROM '.$report_db.'.report_ncn_participation WHERE inside = 1 AND report_tag = "ncn_purdue" AND period = "'.$period.'")';
	} else if ($label == "NCN Other") {
		$sql = 'SELECT tagid FROM '.$report_db.'.report_ncn_tags WHERE report_tag IN (SELECT report_tag FROM '.$report_db.'.report_ncn_participation WHERE inside = 1 AND report_tag <> "ncn_purdue" AND period = "'.$period.'")';
	} else if ($label == "Outside NCN") {
	//	$sql = 'SELECT tagid FROM '.$report_db.'.report_ncn_tags WHERE report_tag IN (SELECT report_tag FROM '.$report_db.'.report_ncn_participation WHERE report_tag <> "ncn_purdue" AND period = "'.$period.'")';
		$sql = 'SELECT tagid FROM '.$report_db.'.report_ncn_tags WHERE report_tag IN (SELECT report_tag FROM '.$report_db.'.report_ncn_participation WHERE report_tag = "ncn_outside" AND period = "'.$period.'")';
	} else {
		return;
	}
	$result = test_execute_query($db_hub, $sql);//mysqli_query($sql, $db_hub);
	if($result) {
		if(mysqli_num_rows($result) > 0) {
   			while($row = mysqli_fetch_row($result)) {
				$tag_list .= $row[0].',';
			}
		}
	} else {
		echo mysqli_error($db_hub)." while executing ".$sql."\n";
	    die;
	}
	$tag_list = rtrim($tag_list, ',');

	return $tag_list;
}

In [7]:
// for the compute years and participate NCN data
function compute_years($db_hub, $label, $statement, $total=0, $report_db){

	//global $cumulative, $report_db;
    //echo "report db is ". $Global['report_db'];
    echo "In compute years report_db is ". $report_db;
	print $label.",";
	$sql = 'SELECT period, startdate, enddate FROM '.$report_db.'.report_periods order by enddate ASC';
	print "sql: ".$sql;
    //test_execute_query($db_hub ,$sql);//mysqli_query($sql, $db_hub);
	$result = test_execute_query($db_hub ,$sql);//mysqli_query($sql, $db_hub);
	if($result) {
		if(mysqli_num_rows($result) > 0) {
   			while($row = mysqli_fetch_row($result)) {
				$period = $row[0];
				$startdate = $row[1];
				if ($cumulative)
					$startdate = '2000-01-01';
				$enddate = $row[2];
				if (!$total) {
					$tags = '';
					$tags = get_ncn_participation($db_hub, $period, $label, $report_db);
					$sql_ = $statement.' AND res.publish_up > "'.$startdate.'" AND res.publish_up < "'.$enddate.'" AND objtags.tagid IN ('.$tags.')';
				} else {
					$sql_ = $statement.' AND res.publish_up > "'.$startdate.'" AND res.publish_up < "'.$enddate.'"';
				}
				$result_ = test_execute_query($db_hub ,$sql_);//mysqli_query($sql_, $db_hub);
				if($result_) {
					if(mysqli_num_rows($result_) > 0) {
   						while($row_ = mysqli_fetch_row($result_)) {
							$items = $row_[0];
						}
					}
				} else {
					echo mysqli_error($db_hub)." while executing ".$sql_."\n";
	    			die;
				}
				if ($items) {
					print $items.",";
				} else {
					print ",";
				}
			}
		}
	} else {
		echo mysqli_error($db_hub)." while executing ".$sql."\n";
	    die;
	}
}

In [8]:
$db_hub = TunneledMySqliConnectionFactory::create_from_environment(3318);
$cumulative = 1;
$res_types = array(7,4,1,6,2,5,3,9,10,39);
$res_tags = array(226,227,25,27);
print "\n,Pre-NCN,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6,Year 7,Year 8, Year 9, Year 10, Year 11, Year 12, Year 13, Year 14, Year 15, Year 16, Year 17, Year 18, Last 12 Months, Year 19 (partial), Cummulative\n";
foreach($res_types as $type) {
    
    echo "Report db is ". $report_db;

	$sql = 'SELECT type FROM nanohub.jos_resource_types where id='.$type;
	$result = test_execute_query($db_hub ,$sql);//mysqli_query($sql, $db_hub);
	if($result) {
		if(mysqli_num_rows($result) > 0) {
   			while($row = mysqli_fetch_row($result)) {
   		   	 	$res_type=$row[0];
			}
		}
	} else {
		echo mysqli_error($db_hub)." while executing ".$sql."\n";
	    die;
	}

	print "\n".$res_type."\n";
	# Special Case used in 2007 report
	// remove published=1 for punch tools
	/*
	if ($type == 7) {
		$published = " AND res.published < 2 ";
	} else {
		$published = " AND res.published=1 ";
	}*/
    
	$published = " AND res.published=1 ";

	# Total
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res WHERE res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND res.type="'.$type.'" ';
	compute_years($db_hub, "Total", $statement,1, $report_db);
	print "\n";
/*
	# NCN Purdue 
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" AND res.type="'.$type.'" ';
	compute_years($db_hub, "NCN Purdue", $statement);
	print "\n";
*/

	/*
	# NCN Other
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid '.$published.' AND res.access!=1 AND res.access!=4 AND res.standalone=1 AND objtags.tbl="resources" AND res.type="'.$type.'" ';
	compute_years($db_hub, "NCN Other", $statement);
	print "\n";
	*/

    /*
	# Outside NCN
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" AND res.type="'.$type.'" ';
	compute_years($db_hub, "Outside NCN", $statement);
	print "\n";
*/
}



,Pre-NCN,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6,Year 7,Year 8, Year 9, Year 10, Year 11, Year 12, Year 13, Year 14, Year 15, Year 16, Year 17, Year 18, Last 12 Months, Year 19 (partial), Cummulative



Report db is nanohub_annualreport



Tools



In compute years report_db is nanohub_annualreport


Total,


sql: SELECT period, startdate, enddate FROM nanohub_annualreport.report_periods order by enddate ASC


mysqli_sql_exception with message 'SELECT command denied to user 'mishr100_ro'@'localhost' for table 'report_periods''

In [9]:
//<?php

# This script generates the table: "Table3.9 - Content Generation Summary"
# Usage: php gen_table_content_generation_summary.php > out_content_generation_summary
# This script generates a csv file which needs to be formatted into the final table: http://vox.hubzero.org/report/ncnreport/2013
# Set parameters between 'TODO-Begin' and 'TODO-End'
# ---------------------------------------------------------------------------------------------------------
/*
require_once('/var/www/tmp/metrics_ncnreport/hub_parameters.php');
require_once('/opt/hubzero/bin/metrics/includes/db_connect.php');
require_once('/opt/hubzero/bin/metrics/includes/func_misc.php');

$db_hub = db_connect('db_hub');
*/
$db_hub = TunneledMySqliConnectionFactory::create_from_environment(3318);
$cumulative = 1;

# ---------------------------------------------------------------------------------------------------------
# TODO-Begin

## List of Resource Types used in this report
// Tools: 7
// Learning Modules: 4
// Online Presentations: 1
// Courses: 6 
// Workshops: 2
// Animations: 5
// Publications: 3
// Downloads: 9
// Notes: 10
// Teaching Materials: 39
// Non-Interative Educational Material: ?
// Non Interactive documents: ?
$res_types = array(7,4,1,6,2,5,3,9,10,39);

## List of Tags used in this report
// Tutorials: 25
// Research Seminars: 27
// Serial Tools: 226
// Parallel Tools: 227
$res_tags = array(226,227,25,27);

# TODO-End
# ---------------------------------------------------------------------------------------------------------

print "\n,Pre-NCN,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6,Year 7,Year 8, Year 9, Year 10, Year 11, Year 12, Year 13, Year 14, Year 15, Year 16, Year 17, Year 18, Last 12 Months, Year 19 (partial), Cummulative\n";
foreach($res_types as $type) {

	$sql = 'SELECT type FROM nanohub.jos_resource_types where id='.$type;
	$result = mysqli_query($sql, $db_hub);
	if($result) {
		if(mysqli_num_rows($result) > 0) {
   			while($row = mysqli_fetch_row($result)) {
   		   	 	$res_type=$row[0];
			}
		}
	} else {
		echo mysqli_error($db_hub)." while executing ".$sql."\n";
	    die;
	}

	print "\n".$res_type."\n";
	# Special Case used in 2007 report
	// remove published=1 for punch tools
	/*
	if ($type == 7) {
		$published = " AND res.published < 2 ";
	} else {
		$published = " AND res.published=1 ";
	}*/
	$published = " AND res.published=1 ";

	# Total
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res WHERE res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND res.type="'.$type.'" ';
	compute_years($db_hub, "Total", $statement,1);
	print "\n";

	# NCN Purdue 
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" AND res.type="'.$type.'" ';
	compute_years($db_hub, "NCN Purdue", $statement);
	print "\n";

	/*
	# NCN Other
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid '.$published.' AND res.access!=1 AND res.access!=4 AND res.standalone=1 AND objtags.tbl="resources" AND res.type="'.$type.'" ';
	compute_years($db_hub, "NCN Other", $statement);
	print "\n";
	*/

	# Outside NCN
	$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" AND res.type="'.$type.'" ';
	compute_years($db_hub, "Outside NCN", $statement);
	print "\n";

}


// -------
// All "and more" Resources 
print "\nTotal 'and more' Content\n";

# Special Case used in 2007 report
#$published = " AND res.published < 2 ";
$published = " AND res.published = 1 ";

# Total
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res WHERE res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND res.type <> 7';
compute_years($db_hub, "Total", $statement,1);
print "\n";

# NCN Purdue
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" AND res.type <> 7 ';
compute_years($db_hub, "NCN Purdue", $statement);
print "\n";

/*
# NCN Other
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid '.$published.' AND res.access!=1 AND res.access!=4 AND res.standalone=1 AND objtags.tbl="resources" AND res.type <> 7';
compute_years($db_hub, "NCN Other", $statement);
print "\n";
*/

# Outside NCN
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" AND res.type <> 7';
compute_years($db_hub, "Outside NCN", $statement);
print "\n\n";

// -------
// All Resources 
print "Total Resources\n";

#$published = " AND res.published < 2 ";
$published = " AND res.published = 1 ";

# Total
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res WHERE res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1';
compute_years($db_hub, "Total", $statement,1);
print "\n";

# NCN Purdue
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" ';
compute_years($db_hub, "NCN Purdue", $statement);
print "\n";

/*
# NCN Other
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid '.$published.' AND res.access!=1 AND res.access!=4 AND res.standalone=1 AND objtags.tbl="resources" ';
compute_years($db_hub, "NCN Other", $statement);
print "\n";
*/

# Outside NCN
$statement = 'SELECT COUNT(DISTINCT(res.id)) FROM nanohub.jos_resources AS res, nanohub.jos_tags_object AS objtags WHERE res.id=objtags.objectid AND res.access!=1 AND res.access!=4 '.$published.' AND res.standalone=1 AND objtags.tbl="resources" ';
compute_years($db_hub, "Outside NCN", $statement);
print "\n\n";

#db_close($db_hub);
// -------


?>


,Pre-NCN,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6,Year 7,Year 8, Year 9, Year 10, Year 11, Year 12, Year 13, Year 14, Year 15, Year 16, Year 17, Year 18, Last 12 Months, Year 19 (partial), Cummulative



PHP Warning:  mysqli_query() expects parameter 1 to be mysqli, string given in /home/suyash/.jupyter-php/pkgs/vendor/litipk/jupyter-php/src/Actions/ExecuteAction.php(115) : eval()'d code on line 43